In [1]:
import pandas as pd

# Load the CSV file
file_path = '/logo2/irfan/Bladder2/files_for_Irfan_global_binning/Top_100_Presumed_cancer_vs_no_pathcr.csv'
data = pd.read_csv(file_path)

top_n = 100
# Extract the top 50 rows and make a copy to avoid SettingWithCopyWarning
top_50 = data.head(top_n).copy()

# Define a function to split the 'Column' values into chromosome, start, and end positions
def split_chromosome_positions(row):
    parts = row.split('_')
    chromosome = parts[0]
    start_position = parts[1]
    end_position = parts[2]
    return pd.Series([chromosome, start_position, end_position])

# Apply the function to each row in the top 50 rows
top_50[['Chromosome', 'Start', 'End']] = top_50['Column'].apply(split_chromosome_positions)

# Display the modified DataFrame

top_50.head(n=10)

,Unnamed: 0,Column,t-statistic,p-value,adjusted_p-value,Chromosome,Start,End
0,2507,chr15_58014600_58114600,-4.122291,0.000113,0.042020,chr15,58014600,58114600
1,22627,chr17_35100000_35200000,-4.052472,0.000144,0.042020,chr17,35100000,35200000
2,9891,chr15_63900000_64000000,-4.024047,0.000158,0.042020,chr15,63900000,64000000
3,9355,chr11_111300000_111400000,4.005788,0.000168,0.042020,chr11,111300000,111400000
4,8413,chr17_47741300_47841300,3.910274,0.000231,0.046233,chr17,47741300,47841300
5,21659,chr19_43300000_43400000,-3.847225,0.000285,0.047447,chr19,43300000,43400000
6,13954,chr4_91200000_91300000,-2.157666,0.034836,0.049230,chr4,91200000,91300000
7,5542,chr5_95170400_95270400,-2.158033,0.034807,0.049230,chr5,95170400,95270400
8,14421,chr1_235000000_235100000,-2.158188,0.034794,0.049230,chr1,235000000,235100000
9,1418,chr10_94400000_94500000,2.158470,0.034771,0.049230,chr10,94400000,94500000


In [2]:
import pybedtools
import pandas as pd

# Assuming your top 50 data is already in 'top_50' DataFrame
# Convert top 50 positions to BED format
top_50_bed = pybedtools.BedTool.from_dataframe(top_50[['Chromosome', 'Start', 'End']])
#print(top_50_bed)  # Check if this prints correctly

# Load your gene list and convert it to BED format
# Replace 'gene_list.csv' with the path to your gene list file
gene_list = pd.read_csv('/logo2/irfan/Reference_like_Data/Ensembl/Ensemble_unique_ProteinCodig_genes_sorted.bed', sep='\t')
gene_list_bed = pybedtools.BedTool.from_dataframe(gene_list)
#print(gene_list_bed.head())  # Check if this prints correctly


# Sort the top_50_bed object
top_50_bed_sorted = top_50_bed.sort()

# Now you can use the sorted object for the closest operation
closest_genes = top_50_bed_sorted.closest(gene_list_bed, d=True, stream=True)



In [3]:
# Save the BedTool output to a temporary file
tempfile = closest_genes.saveas().fn

# Read the temporary file into a pandas DataFrame
closest_genes_df = pd.read_csv(tempfile, sep='\t', header=None, names=['chr', 'start', 'end', 'gene_chr', 'gene_start', 'gene_end', 'gene_name', 'distance'])

# Display the head of the DataFrame to check
print(closest_genes_df.head())

    chr      start        end gene_chr  gene_start   gene_end gene_name  \
0  chr1   45800000   45900000     chr1    45786987   46036122     MAST2   
1  chr1   80900000   81000000     chr1    81306147   81993932    ADGRL2   
2  chr1   82900000   83000000     chr1    83865024   83999150     TTLL7   
3  chr1  116300000  116400000     chr1   116372668  116410261    ATP1A1   
4  chr1  167672400  167772400     chr1   167630093  167708696     RCSD1   

   distance  
0         0  
1    306148  
2    865025  
3         0  
4         0  


In [4]:
closest_genes_df.shape

(136, 8)

In [5]:
closest_genes_df.to_csv(file_path+"_closest_gene"+str(top_n)+".txt",sep='\t')